# Data Science aplicada em Logística 

Uma transportadora possui diversos veículos que são usados para entregas. Cada veículo realiza diversas entregas por dia e o horário exato de cada entrega é registrado. A empresa tenta otimizar o percurso de cada veículo fazendo com que um mesmo veículo faça todas as entregas em cada região, reduzindo assim o tempo entre uma entrega e outra.

Os dados usados neste trabalho são fictícios e o dataset tem 3 colunas: id do veículo, a entrega que foi realizada e o horário. Usando Spark SQL faremos uma série de análises nos dados a fim de verificar como está a performance da logística de entrega da empresa.

O dataset possui poucos registros para que as consultas sejam realizadas rapidamente, pois o mini-projeto traz diversos exemplos de funções e notações do Spark SQL.

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.7


In [2]:
# Importa o findspark e inicializa
import findspark
findspark.init()

In [3]:
# Imports
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col
from pyspark.sql.functions import row_number
from pyspark.sql.functions import lead  
from pyspark.sql.functions import min, max
from pyspark.sql.functions import unix_timestamp

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Lucas Castro" --iversions

Author: Lucas Castro

pyspark  : 3.3.0
findspark: 2.0.1



## Preparando o Ambiente Spark


In [5]:
# Criando o Spark Context
sc = SparkContext(appName = "Mini-Projeto4")

22/09/15 14:09:54 WARN Utils: Your hostname, pcastro resolves to a loopback address: 127.0.1.1; using 192.168.0.20 instead (on interface enp3s0)
22/09/15 14:09:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 14:09:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/15 14:09:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
# Cria a sessão
spark = SparkSession.builder.getOrCreate()

In [7]:
spark

## Carregando os Dados Como Dataframe do Spark

In [8]:
# Nome do arquivo
arquivo = 'dados/dataset.txt'

In [9]:
# Carrega como dataframe do Spark
# Não usaremos Pandas pois não vamos fazer análise exploratória de dados
df = spark.read.csv(arquivo, header = True)

In [10]:
type(df)

pyspark.sql.dataframe.DataFrame

In [11]:
df.show(5)

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
+----------+---------+-------+
only showing top 5 rows



## Criando Tabela Temporária

Criamos uma tabela temporária para executar consultas SQL nos dados. A tabela temporária existe somente nesta sessão.

In [12]:
# Cria tabela temporária
df.createOrReplaceTempView("tb_logistica")

## Executando Queries SQL

In [13]:
# Verificando as colunas da tabela
spark.sql("SHOW COLUMNS FROM tb_logistica").show()

+----------+
|  col_name|
+----------+
|id_veiculo|
|   entrega|
|   horario|
+----------+



In [14]:
# Visualizando os 5 primeiros registros
spark.sql("SELECT * FROM tb_logistica LIMIT 5").show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
+----------+---------+-------+



In [15]:
# Describe da tabela
spark.sql("DESCRIBE tb_logistica").show()

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|id_veiculo|   string|   null|
|   entrega|   string|   null|
|   horario|   string|   null|
+----------+---------+-------+



## Queries SQL x Dot Notation no Spark SQL

In [16]:
# Query SQL - Dependendo do volume de dados isso se torna inviável pois os dados ficam duplicados(usa a tabela temporária)
spark.sql('SELECT id_veiculo AS veiculo, entrega FROM tb_logistica LIMIT 5').show()

+-------+---------+
|veiculo|  entrega|
+-------+---------+
|    298|Entrega 1|
|    298|Entrega 2|
|    298|Entrega 3|
|    298|Entrega 4|
|    298|Entrega 5|
+-------+---------+



In [17]:
# Dot Notation- Não usa a tabela temporária, é otimizado para ambiente distribuído
df.select(col('id_veiculo').alias('veiculo'), 'entrega').limit(5).show()

+-------+---------+
|veiculo|  entrega|
+-------+---------+
|    298|Entrega 1|
|    298|Entrega 2|
|    298|Entrega 3|
|    298|Entrega 4|
|    298|Entrega 5|
+-------+---------+



## Usando Funções SQL do Spark SQL

Embora seja mais fácil usar direto Linguagem SQL, as funções do Spark SQL são otimizadas para o trabalho em ambiente distribuído. Se estiver com problemas de performance ao processar grandes conjuntos de dados, faça o teste com SQL e com o uso de funções e compare os resultados.

In [18]:
# Após o nome do dataframe, digite . (ponto) e pressione a tecla tab para visualizar as funções (métodos) disponíveis
df

DataFrame[id_veiculo: string, entrega: string, horario: string]

In [19]:
# Colunas do dataframe
df.columns

['id_veiculo', 'entrega', 'horario']

In [20]:
# Podemos converter um Spark DataFrame para um Pandas DataFrame (e assim usar os métodos do Pandas)
pandasDF = df.toPandas()

In [21]:
type(pandasDF)

pandas.core.frame.DataFrame

In [22]:
print(pandasDF)

   id_veiculo    entrega horario
0         298  Entrega 1   7:58a
1         298  Entrega 2   8:04a
2         298  Entrega 3   8:17a
3         298  Entrega 4   8:28a
4         298  Entrega 5   8:33a
5         298  Entrega 6   8:39a
6         298  Entrega 7   9:07a
7         315  Entrega 1   6:05a
8         315  Entrega 2   6:14a
9         315  Entrega 3   6:24a
10        315  Entrega 4   6:38a
11        315  Entrega 5   6:45a
12        315  Entrega 6   6:56a
13        315  Entrega 7   7:32a
14        457  Entrega 1   5:04a
15        457  Entrega 2   5:13a
16        457  Entrega 3   5:27a
17        457  Entrega 4   5:39a
18        457  Entrega 5   5:47a
19        457  Entrega 6   6:21a
20        457  Entrega 7   6:38a


### Métodos Select e Collect

> Função select()

In [23]:
# Selecionando dados de 2 colunas
df.select('id_veiculo', 'entrega').show(10)

+----------+---------+
|id_veiculo|  entrega|
+----------+---------+
|       298|Entrega 1|
|       298|Entrega 2|
|       298|Entrega 3|
|       298|Entrega 4|
|       298|Entrega 5|
|       298|Entrega 6|
|       298|Entrega 7|
|       315|Entrega 1|
|       315|Entrega 2|
|       315|Entrega 3|
+----------+---------+
only showing top 10 rows



In [24]:
# Alternativamente podemos usar esta notação
df.select(df.id_veiculo, df.entrega).show(10)

+----------+---------+
|id_veiculo|  entrega|
+----------+---------+
|       298|Entrega 1|
|       298|Entrega 2|
|       298|Entrega 3|
|       298|Entrega 4|
|       298|Entrega 5|
|       298|Entrega 6|
|       298|Entrega 7|
|       315|Entrega 1|
|       315|Entrega 2|
|       315|Entrega 3|
+----------+---------+
only showing top 10 rows



In [25]:
# A função col é outra alternativa
from pyspark.sql.functions import col
df.select(col('id_veiculo'), col('entrega')).show(10)

+----------+---------+
|id_veiculo|  entrega|
+----------+---------+
|       298|Entrega 1|
|       298|Entrega 2|
|       298|Entrega 3|
|       298|Entrega 4|
|       298|Entrega 5|
|       298|Entrega 6|
|       298|Entrega 7|
|       315|Entrega 1|
|       315|Entrega 2|
|       315|Entrega 3|
+----------+---------+
only showing top 10 rows



In [26]:
# Podemos selecionar todas as colunas do dataframe cujos nomes estejam em uma lista
nomes_colunas = ["id_veiculo", "entrega"]
df.select(*nomes_colunas).show(10)

+----------+---------+
|id_veiculo|  entrega|
+----------+---------+
|       298|Entrega 1|
|       298|Entrega 2|
|       298|Entrega 3|
|       298|Entrega 4|
|       298|Entrega 5|
|       298|Entrega 6|
|       298|Entrega 7|
|       315|Entrega 1|
|       315|Entrega 2|
|       315|Entrega 3|
+----------+---------+
only showing top 10 rows



In [27]:
# Mesmo exemplo anterior mas agora com list comprehension
df.select([coluna for coluna in nomes_colunas]).show()

+----------+---------+
|id_veiculo|  entrega|
+----------+---------+
|       298|Entrega 1|
|       298|Entrega 2|
|       298|Entrega 3|
|       298|Entrega 4|
|       298|Entrega 5|
|       298|Entrega 6|
|       298|Entrega 7|
|       315|Entrega 1|
|       315|Entrega 2|
|       315|Entrega 3|
|       315|Entrega 4|
|       315|Entrega 5|
|       315|Entrega 6|
|       315|Entrega 7|
|       457|Entrega 1|
|       457|Entrega 2|
|       457|Entrega 3|
|       457|Entrega 4|
|       457|Entrega 5|
|       457|Entrega 6|
+----------+---------+
only showing top 20 rows



In [28]:
# Podemos renomear colunas para facilitar a consulta aos dados
df.select('id_veiculo', 'entrega').withColumnRenamed('id_veiculo', 'veiculo').show(10)

+-------+---------+
|veiculo|  entrega|
+-------+---------+
|    298|Entrega 1|
|    298|Entrega 2|
|    298|Entrega 3|
|    298|Entrega 4|
|    298|Entrega 5|
|    298|Entrega 6|
|    298|Entrega 7|
|    315|Entrega 1|
|    315|Entrega 2|
|    315|Entrega 3|
+-------+---------+
only showing top 10 rows



In [29]:
# Também podemos usar um alias para renomear coluna
df.select(col('id_veiculo').alias('veiculo'), 'entrega').show(10)

+-------+---------+
|veiculo|  entrega|
+-------+---------+
|    298|Entrega 1|
|    298|Entrega 2|
|    298|Entrega 3|
|    298|Entrega 4|
|    298|Entrega 5|
|    298|Entrega 6|
|    298|Entrega 7|
|    315|Entrega 1|
|    315|Entrega 2|
|    315|Entrega 3|
+-------+---------+
only showing top 10 rows



In [30]:
df.show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
|       298|Entrega 6|  8:39a|
|       298|Entrega 7|  9:07a|
|       315|Entrega 1|  6:05a|
|       315|Entrega 2|  6:14a|
|       315|Entrega 3|  6:24a|
|       315|Entrega 4|  6:38a|
|       315|Entrega 5|  6:45a|
|       315|Entrega 6|  6:56a|
|       315|Entrega 7|  7:32a|
|       457|Entrega 1|  5:04a|
|       457|Entrega 2|  5:13a|
|       457|Entrega 3|  5:27a|
|       457|Entrega 4|  5:39a|
|       457|Entrega 5|  5:47a|
|       457|Entrega 6|  6:21a|
+----------+---------+-------+
only showing top 20 rows



In [31]:
# Selecionando colunas pelo índice
# A partir da coluna de índice 2 retorne as 3 primeiras linhas
df.select(df.columns[2:]).show(3)

+-------+
|horario|
+-------+
|  7:58a|
|  8:04a|
|  8:17a|
+-------+
only showing top 3 rows



In [32]:
# Selecionando colunas através de expressões regulares
# https://docs.python.org/3.9/library/re.html
df.select(df.colRegex("`^.*Entrega*`")).show()

+---------+
|  entrega|
+---------+
|Entrega 1|
|Entrega 2|
|Entrega 3|
|Entrega 4|
|Entrega 5|
|Entrega 6|
|Entrega 7|
|Entrega 1|
|Entrega 2|
|Entrega 3|
|Entrega 4|
|Entrega 5|
|Entrega 6|
|Entrega 7|
|Entrega 1|
|Entrega 2|
|Entrega 3|
|Entrega 4|
|Entrega 5|
|Entrega 6|
+---------+
only showing top 20 rows



In [33]:
# O dataframe original segue intacto
df.show(21)

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
|       298|Entrega 6|  8:39a|
|       298|Entrega 7|  9:07a|
|       315|Entrega 1|  6:05a|
|       315|Entrega 2|  6:14a|
|       315|Entrega 3|  6:24a|
|       315|Entrega 4|  6:38a|
|       315|Entrega 5|  6:45a|
|       315|Entrega 6|  6:56a|
|       315|Entrega 7|  7:32a|
|       457|Entrega 1|  5:04a|
|       457|Entrega 2|  5:13a|
|       457|Entrega 3|  5:27a|
|       457|Entrega 4|  5:39a|
|       457|Entrega 5|  5:47a|
|       457|Entrega 6|  6:21a|
|       457|Entrega 7|  6:38a|
+----------+---------+-------+



> Vejamos agora a função collect()

In [34]:
# Retornando cada linha do dataframe como um objeto do tipo Row
df.collect()

[Row(id_veiculo='298', entrega='Entrega 1', horario='7:58a'),
 Row(id_veiculo='298', entrega='Entrega 2', horario='8:04a'),
 Row(id_veiculo='298', entrega='Entrega 3', horario='8:17a'),
 Row(id_veiculo='298', entrega='Entrega 4', horario='8:28a'),
 Row(id_veiculo='298', entrega='Entrega 5', horario='8:33a'),
 Row(id_veiculo='298', entrega='Entrega 6', horario='8:39a'),
 Row(id_veiculo='298', entrega='Entrega 7', horario='9:07a'),
 Row(id_veiculo='315', entrega='Entrega 1', horario='6:05a'),
 Row(id_veiculo='315', entrega='Entrega 2', horario='6:14a'),
 Row(id_veiculo='315', entrega='Entrega 3', horario='6:24a'),
 Row(id_veiculo='315', entrega='Entrega 4', horario='6:38a'),
 Row(id_veiculo='315', entrega='Entrega 5', horario='6:45a'),
 Row(id_veiculo='315', entrega='Entrega 6', horario='6:56a'),
 Row(id_veiculo='315', entrega='Entrega 7', horario='7:32a'),
 Row(id_veiculo='457', entrega='Entrega 1', horario='5:04a'),
 Row(id_veiculo='457', entrega='Entrega 2', horario='5:13a'),
 Row(id_

In [35]:
# O método collect() retorna uma lista de linhas
new_df = df.collect()
type(new_df)

list

In [36]:
# Podemos "fatiar" as estruturas retornadas por collect()
# Neste caso retornamos o elemento da primeira linha e terceira coluna
df.collect()[0][2]

'7:58a'

In [37]:
# Como collect() retorna uma lista, podemos percorrer a lista com um loop e concatenar as colunas, por exemplo
for row in df.collect():
    print(row['id_veiculo'] + "," + str(row['entrega']))

298,Entrega 1
298,Entrega 2
298,Entrega 3
298,Entrega 4
298,Entrega 5
298,Entrega 6
298,Entrega 7
315,Entrega 1
315,Entrega 2
315,Entrega 3
315,Entrega 4
315,Entrega 5
315,Entrega 6
315,Entrega 7
457,Entrega 1
457,Entrega 2
457,Entrega 3
457,Entrega 4
457,Entrega 5
457,Entrega 6
457,Entrega 7


In [38]:
# Podemos ainda combinar select e collect
# Filtramos colunas com select e filtramos linhas com collect
dataCollect = df.select("id_veiculo").collect()[0:4][:]
dataCollect

[Row(id_veiculo='298'),
 Row(id_veiculo='298'),
 Row(id_veiculo='298'),
 Row(id_veiculo='298')]

In [39]:
# Podemos extrair primeiro uma amostra do dataframe e então coletar o resultado
df.sample(0.6).collect()

[Row(id_veiculo='298', entrega='Entrega 2', horario='8:04a'),
 Row(id_veiculo='298', entrega='Entrega 4', horario='8:28a'),
 Row(id_veiculo='298', entrega='Entrega 5', horario='8:33a'),
 Row(id_veiculo='298', entrega='Entrega 7', horario='9:07a'),
 Row(id_veiculo='315', entrega='Entrega 1', horario='6:05a'),
 Row(id_veiculo='315', entrega='Entrega 2', horario='6:14a'),
 Row(id_veiculo='315', entrega='Entrega 3', horario='6:24a'),
 Row(id_veiculo='315', entrega='Entrega 5', horario='6:45a'),
 Row(id_veiculo='315', entrega='Entrega 6', horario='6:56a'),
 Row(id_veiculo='315', entrega='Entrega 7', horario='7:32a'),
 Row(id_veiculo='457', entrega='Entrega 1', horario='5:04a'),
 Row(id_veiculo='457', entrega='Entrega 2', horario='5:13a'),
 Row(id_veiculo='457', entrega='Entrega 3', horario='5:27a'),
 Row(id_veiculo='457', entrega='Entrega 4', horario='5:39a')]

## Métodos Filter e Where

In [40]:
# Podemos filtrar os dados retornados com a função filter()
df.filter("entrega == 'Entrega 2'").show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 2|  8:04a|
|       315|Entrega 2|  6:14a|
|       457|Entrega 2|  5:13a|
+----------+---------+-------+



In [41]:
# Podemos filtrar os dados retornados com a função filter() usando a negação
df.filter("entrega != 'Entrega 2'").show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
|       298|Entrega 6|  8:39a|
|       298|Entrega 7|  9:07a|
|       315|Entrega 1|  6:05a|
|       315|Entrega 3|  6:24a|
|       315|Entrega 4|  6:38a|
|       315|Entrega 5|  6:45a|
|       315|Entrega 6|  6:56a|
|       315|Entrega 7|  7:32a|
|       457|Entrega 1|  5:04a|
|       457|Entrega 3|  5:27a|
|       457|Entrega 4|  5:39a|
|       457|Entrega 5|  5:47a|
|       457|Entrega 6|  6:21a|
|       457|Entrega 7|  6:38a|
+----------+---------+-------+



In [42]:
# Podemos filtrar usando múltiplas condições
df.filter((df.entrega == "Entrega 2") & (df.id_veiculo == 298)).show()  

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 2|  8:04a|
+----------+---------+-------+



In [43]:
# Filtro baseado em lista
lista_id_veiculos = [298, 300, 400]
df.filter(df.id_veiculo.isin(lista_id_veiculos)).show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
|       298|Entrega 6|  8:39a|
|       298|Entrega 7|  9:07a|
+----------+---------+-------+



In [44]:
# Alguma entrega ocorreu no minuto 38 de qualquer hora?
df.filter(df.horario.like("%38%")).show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       315|Entrega 4|  6:38a|
|       457|Entrega 7|  6:38a|
+----------+---------+-------+



In [45]:
# Podemos filtrar os dados retornados com a função where()
df.where("entrega == 'Entrega 2'").show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 2|  8:04a|
|       315|Entrega 2|  6:14a|
|       457|Entrega 2|  5:13a|
+----------+---------+-------+



In [46]:
# Podemos filtrar os dados retornados com a função where()
df.where("id_veiculo > 400").show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       457|Entrega 1|  5:04a|
|       457|Entrega 2|  5:13a|
|       457|Entrega 3|  5:27a|
|       457|Entrega 4|  5:39a|
|       457|Entrega 5|  5:47a|
|       457|Entrega 6|  6:21a|
|       457|Entrega 7|  6:38a|
+----------+---------+-------+



In [47]:
# Where baseado em lista
lista_id_veiculos = [298, 300, 400]
df.where(df.id_veiculo.isin(lista_id_veiculos)).show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
|       298|Entrega 6|  8:39a|
|       298|Entrega 7|  9:07a|
+----------+---------+-------+



### Métodos Order By e Sort


In [48]:
# Ordenando a seleção das linhas
df.sort("horario", "entrega").show(10)

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       457|Entrega 1|  5:04a|
|       457|Entrega 2|  5:13a|
|       457|Entrega 3|  5:27a|
|       457|Entrega 4|  5:39a|
|       457|Entrega 5|  5:47a|
|       315|Entrega 1|  6:05a|
|       315|Entrega 2|  6:14a|
|       457|Entrega 6|  6:21a|
|       315|Entrega 3|  6:24a|
|       315|Entrega 4|  6:38a|
+----------+---------+-------+
only showing top 10 rows



In [49]:
# Mesmo resultado anterior mas com a função col()
df.sort(col("horario"), col("entrega")).show(10)

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       457|Entrega 1|  5:04a|
|       457|Entrega 2|  5:13a|
|       457|Entrega 3|  5:27a|
|       457|Entrega 4|  5:39a|
|       457|Entrega 5|  5:47a|
|       315|Entrega 1|  6:05a|
|       315|Entrega 2|  6:14a|
|       457|Entrega 6|  6:21a|
|       315|Entrega 3|  6:24a|
|       315|Entrega 4|  6:38a|
+----------+---------+-------+
only showing top 10 rows



In [50]:
# Ou usamos Order By
df.orderBy(col("horario"), col("entrega")).show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       457|Entrega 1|  5:04a|
|       457|Entrega 2|  5:13a|
|       457|Entrega 3|  5:27a|
|       457|Entrega 4|  5:39a|
|       457|Entrega 5|  5:47a|
|       315|Entrega 1|  6:05a|
|       315|Entrega 2|  6:14a|
|       457|Entrega 6|  6:21a|
|       315|Entrega 3|  6:24a|
|       315|Entrega 4|  6:38a|
|       457|Entrega 7|  6:38a|
|       315|Entrega 5|  6:45a|
|       315|Entrega 6|  6:56a|
|       315|Entrega 7|  7:32a|
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
|       298|Entrega 6|  8:39a|
+----------+---------+-------+
only showing top 20 rows



In [51]:
# Ordena o resultado em ordem decrescente 
df.sort(df.horario.desc(), df.entrega.desc()).show(10)

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 7|  9:07a|
|       298|Entrega 6|  8:39a|
|       298|Entrega 5|  8:33a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 1|  7:58a|
|       315|Entrega 7|  7:32a|
|       315|Entrega 6|  6:56a|
|       315|Entrega 5|  6:45a|
+----------+---------+-------+
only showing top 10 rows



In [52]:
# Lembre-se que podemos usar SQL
spark.sql("select id_veiculo, entrega, horario from tb_logistica ORDER BY horario desc").show(10)

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 7|  9:07a|
|       298|Entrega 6|  8:39a|
|       298|Entrega 5|  8:33a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 1|  7:58a|
|       315|Entrega 7|  7:32a|
|       315|Entrega 6|  6:56a|
|       315|Entrega 5|  6:45a|
+----------+---------+-------+
only showing top 10 rows



### Métodos Map, flatMap e Explode


Função map( ) em Python: https://docs.python.org/3.9/library/functions.html#map

Função map( ) do Pandas: https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html

Função map( ) do Spark: https://spark.apache.org/docs/latest/api/sql/index.html#map

In [53]:
# Maps são aplicados em RDDs e por isso precisamos converter o dataframe para RDD
# O método Map retorna um RDD e por isso temos que converter de volta para dataframe
rdd2 = df.rdd.map(lambda x: (x[0] + "," + x[1], x[2]))  
df2 = rdd2.toDF(["novo_id", "entrega"])
df2.show()

+-------------+-------+
|      novo_id|entrega|
+-------------+-------+
|298,Entrega 1|  7:58a|
|298,Entrega 2|  8:04a|
|298,Entrega 3|  8:17a|
|298,Entrega 4|  8:28a|
|298,Entrega 5|  8:33a|
|298,Entrega 6|  8:39a|
|298,Entrega 7|  9:07a|
|315,Entrega 1|  6:05a|
|315,Entrega 2|  6:14a|
|315,Entrega 3|  6:24a|
|315,Entrega 4|  6:38a|
|315,Entrega 5|  6:45a|
|315,Entrega 6|  6:56a|
|315,Entrega 7|  7:32a|
|457,Entrega 1|  5:04a|
|457,Entrega 2|  5:13a|
|457,Entrega 3|  5:27a|
|457,Entrega 4|  5:39a|
|457,Entrega 5|  5:47a|
|457,Entrega 6|  6:21a|
+-------------+-------+
only showing top 20 rows



In [54]:
df.columns

['id_veiculo', 'entrega', 'horario']

In [55]:
# Mesmo exemplo anterior mas usando o nome da coluna e não o índice
rdd2 = df.rdd.map(lambda x: (x['id_veiculo'] + "," + x['entrega'], x['horario']))  
df2 = rdd2.toDF(["novo_id", "entrega"])
df2.show()

+-------------+-------+
|      novo_id|entrega|
+-------------+-------+
|298,Entrega 1|  7:58a|
|298,Entrega 2|  8:04a|
|298,Entrega 3|  8:17a|
|298,Entrega 4|  8:28a|
|298,Entrega 5|  8:33a|
|298,Entrega 6|  8:39a|
|298,Entrega 7|  9:07a|
|315,Entrega 1|  6:05a|
|315,Entrega 2|  6:14a|
|315,Entrega 3|  6:24a|
|315,Entrega 4|  6:38a|
|315,Entrega 5|  6:45a|
|315,Entrega 6|  6:56a|
|315,Entrega 7|  7:32a|
|457,Entrega 1|  5:04a|
|457,Entrega 2|  5:13a|
|457,Entrega 3|  5:27a|
|457,Entrega 4|  5:39a|
|457,Entrega 5|  5:47a|
|457,Entrega 6|  6:21a|
+-------------+-------+
only showing top 20 rows



In [56]:
# Criamos uma função que manipula as colunas
def manipula_colunas(x):
    coluna1 = x.id_veiculo
    coluna2 = x.entrega
    novo_id = coluna1 + "-" + coluna2
    coluna3 = x.horario
    return (novo_id, coluna3)

In [57]:
# Usamos a função map para aplicar a função lambda (anônima), que aplica a função manipula_colunas a cada linha do RDD
rdd2 = df.rdd.map(lambda x: manipula_colunas(x))

In [58]:
# Collect no RDD
rdd2.collect()

[('298-Entrega 1', '7:58a'),
 ('298-Entrega 2', '8:04a'),
 ('298-Entrega 3', '8:17a'),
 ('298-Entrega 4', '8:28a'),
 ('298-Entrega 5', '8:33a'),
 ('298-Entrega 6', '8:39a'),
 ('298-Entrega 7', '9:07a'),
 ('315-Entrega 1', '6:05a'),
 ('315-Entrega 2', '6:14a'),
 ('315-Entrega 3', '6:24a'),
 ('315-Entrega 4', '6:38a'),
 ('315-Entrega 5', '6:45a'),
 ('315-Entrega 6', '6:56a'),
 ('315-Entrega 7', '7:32a'),
 ('457-Entrega 1', '5:04a'),
 ('457-Entrega 2', '5:13a'),
 ('457-Entrega 3', '5:27a'),
 ('457-Entrega 4', '5:39a'),
 ('457-Entrega 5', '5:47a'),
 ('457-Entrega 6', '6:21a'),
 ('457-Entrega 7', '6:38a')]

In [59]:
# O método flatMap requer uma lista no formato RDD
# Vamos criar uma lista
data = ["A Universidade Federal da Paraíba",
        "oferece cursos realmente incríveis",
        "orientados às necessidades",
        "do mercado de trabalho",
        "e tudo mostrado passo a passo"]

In [60]:
# Convertemos a lista em um RDD
rdd = spark.sparkContext.parallelize(data)
type(rdd)

pyspark.rdd.RDD

In [61]:
# Imprime os elementos do RDD
for element in rdd.collect():
    print(element)

A Universidade Federal da Paraíba
oferece cursos realmente incríveis
orientados às necessidades
do mercado de trabalho
e tudo mostrado passo a passo


In [62]:
# Agora aplicamos o flatMap, que cria outro RDD  
rdd2 = rdd.flatMap(lambda x: x.split(" "))

In [63]:
# Imprime os elementos do RDD
for element in rdd2.collect():
    print(element)

A
Universidade
Federal
da
Paraíba
oferece
cursos
realmente
incríveis
orientados
às
necessidades
do
mercado
de
trabalho
e
tudo
mostrado
passo
a
passo


In [64]:
# Explode deve receber uma lista como argumento, mas essa lista pode estar em uma coluna de um dataframe
from pyspark.sql.functions import explode

In [65]:
# Cria uma lista
array_estudantes = [('Bob', ['Python', 'R', 'Scala']),
                    ('Maria', ['Java','Julia']),
                    ('Zico', ['JavaScript', '']),
                    ('Ana', [None, None])]

In [66]:
type(array_estudantes)

list

In [67]:
# Converte a lista para dataframe
df_estudantes = spark.createDataFrame(data = array_estudantes, schema = ['aluno', 'linguagem'])

In [68]:
type(df_estudantes)

pyspark.sql.dataframe.DataFrame

In [69]:
# Select com explode
df2 = df_estudantes.select(df_estudantes.aluno, explode(df_estudantes.linguagem))
df2.printSchema()
df2.show()

root
 |-- aluno: string (nullable = true)
 |-- col: string (nullable = true)

+-----+----------+
|aluno|       col|
+-----+----------+
|  Bob|    Python|
|  Bob|         R|
|  Bob|     Scala|
|Maria|      Java|
|Maria|     Julia|
| Zico|JavaScript|
| Zico|          |
|  Ana|      null|
|  Ana|      null|
+-----+----------+



In [70]:
df.columns

['id_veiculo', 'entrega', 'horario']

In [71]:
# Foreach
df.foreach(lambda x: print(x["id_veiculo"] + "," + x["entrega"] + "," + x["horario"])) 

298,Entrega 1,7:58a
298,Entrega 2,8:04a
298,Entrega 3,8:17a
298,Entrega 4,8:28a
298,Entrega 5,8:33a
298,Entrega 6,8:39a
298,Entrega 7,9:07a
315,Entrega 1,6:05a
315,Entrega 2,6:14a
315,Entrega 3,6:24a
315,Entrega 4,6:38a
315,Entrega 5,6:45a
315,Entrega 6,6:56a
315,Entrega 7,7:32a
457,Entrega 1,5:04a
457,Entrega 2,5:13a
457,Entrega 3,5:27a
457,Entrega 4,5:39a
457,Entrega 5,5:47a
457,Entrega 6,6:21a
457,Entrega 7,6:38a


## Agregação com Spark SQL

> Agregação com Funções do Spark SQL

In [72]:
df.columns

['id_veiculo', 'entrega', 'horario']

In [73]:
df.groupBy("id_veiculo").count().show()

+----------+-----+
|id_veiculo|count|
+----------+-----+
|       298|    7|
|       457|    7|
|       315|    7|
+----------+-----+



In [74]:
# A linha de código abaixo não funciona. LEIA A MENSAGEM DE ERRO!!!
# df.groupBy("id_veiculo").min("horario").show()

In [75]:
df.groupBy('id_veiculo').agg({'horario':'min'}).show()

+----------+------------+
|id_veiculo|min(horario)|
+----------+------------+
|       298|       7:58a|
|       315|       6:05a|
|       457|       5:04a|
+----------+------------+



In [76]:
df.groupBy('id_veiculo').agg({'horario':'max'}).show()

+----------+------------+
|id_veiculo|max(horario)|
+----------+------------+
|       298|       9:07a|
|       315|       7:32a|
|       457|       6:38a|
+----------+------------+



In [77]:
df.groupBy('id_veiculo').agg({'horario':'count'}).show()

+----------+--------------+
|id_veiculo|count(horario)|
+----------+--------------+
|       298|             7|
|       457|             7|
|       315|             7|
+----------+--------------+



In [78]:
df.groupBy('id_veiculo').agg({'horario':'count'}).withColumnRenamed('count(horario)', 'numero_entregas').show()

+----------+---------------+
|id_veiculo|numero_entregas|
+----------+---------------+
|       298|              7|
|       457|              7|
|       315|              7|
+----------+---------------+



In [79]:
df.groupBy('id_veiculo').agg({'horario':'min'}).withColumnRenamed('min(horario)', 'hora_primeira_entrega').show()

+----------+---------------------+
|id_veiculo|hora_primeira_entrega|
+----------+---------------------+
|       298|                7:58a|
|       315|                6:05a|
|       457|                5:04a|
+----------+---------------------+



> Agregação com Queries SQL

In [80]:
# Usando função SQL do SparkSQL
df.groupBy("id_veiculo").count().withColumnRenamed('count', 'numero_entregas').show()

+----------+---------------+
|id_veiculo|numero_entregas|
+----------+---------------+
|       298|              7|
|       457|              7|
|       315|              7|
+----------+---------------+



In [81]:
# Define a query
# Número de entregas por id_veiculo
query = """
SELECT id_veiculo, COUNT(*) AS numero_entregas
FROM tb_logistica
GROUP BY id_veiculo
"""

In [82]:
# Executa a query
spark.sql(query).show()

+----------+---------------+
|id_veiculo|numero_entregas|
+----------+---------------+
|       298|              7|
|       457|              7|
|       315|              7|
+----------+---------------+



In [83]:
# Define a query
# Primeiro (menor) e último (maior) horário de entrega por id_veiculo
query = """
SELECT id_veiculo, MIN(horario) AS hora_primeira_entrega, MAX(horario) AS hora_ultima_entrega
FROM tb_logistica
GROUP BY id_veiculo
"""

In [84]:
# Executa a query
spark.sql(query).show()

+----------+---------------------+-------------------+
|id_veiculo|hora_primeira_entrega|hora_ultima_entrega|
+----------+---------------------+-------------------+
|       298|                7:58a|              9:07a|
|       315|                6:05a|              7:32a|
|       457|                5:04a|              6:38a|
+----------+---------------------+-------------------+



In [85]:
# Define a query
# Número de entregas por horário para o id_veiculo 298
query = """
SELECT horario, COUNT(*) AS hora_ultima_entrega
FROM tb_logistica
WHERE id_veiculo = 298
GROUP BY horario
"""

In [86]:
# Executa a query
spark.sql(query).show()

+-------+-------------------+
|horario|hora_ultima_entrega|
+-------+-------------------+
|  8:33a|                  1|
|  8:28a|                  1|
|  8:17a|                  1|
|  8:39a|                  1|
|  8:04a|                  1|
|  7:58a|                  1|
|  9:07a|                  1|
+-------+-------------------+



In [87]:
# Define a query
# Número de entregas por horário
query = """
SELECT horario, COUNT(*) AS numero_entregas
FROM tb_logistica
GROUP BY horario
"""

In [88]:
# Executa a query
spark.sql(query).show()

+-------+---------------+
|horario|numero_entregas|
+-------+---------------+
|  8:33a|              1|
|  6:45a|              1|
|  8:28a|              1|
|  5:39a|              1|
|  8:17a|              1|
|  8:39a|              1|
|  7:32a|              1|
|  6:56a|              1|
|  5:04a|              1|
|  6:14a|              1|
|  5:13a|              1|
|  8:04a|              1|
|  6:05a|              1|
|  5:27a|              1|
|  6:24a|              1|
|  7:58a|              1|
|  5:47a|              1|
|  6:38a|              2|
|  9:07a|              1|
|  6:21a|              1|
+-------+---------------+



In [89]:
# Define a query
# Horário que teve mais de uma entrega
query = """
SELECT horario, COUNT(*) AS hora_ultima_entrega
FROM tb_logistica
GROUP BY horario
HAVING COUNT(*) > 1
"""

In [90]:
# Executa a query
spark.sql(query).show()

+-------+-------------------+
|horario|hora_ultima_entrega|
+-------+-------------------+
|  6:38a|                  2|
+-------+-------------------+



> Podemos ainda fazer Pivot de um dataframe

In [91]:
# Lista de horários de entregas
lista_horarios = ['5:13a', '6:38a', '7:32a', '8:04a', '9:07a']

In [92]:
# Testamos o filtro
df.filter(df.horario.isin(lista_horarios)).show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 2|  8:04a|
|       298|Entrega 7|  9:07a|
|       315|Entrega 4|  6:38a|
|       315|Entrega 7|  7:32a|
|       457|Entrega 2|  5:13a|
|       457|Entrega 7|  6:38a|
+----------+---------+-------+



In [93]:
# Pivot é uma função de agregação no Spark
df_pivot = df.filter(df.horario.isin(lista_horarios)).groupBy("id_veiculo").pivot("horario").count()

In [94]:
df_pivot.show()

+----------+-----+-----+-----+-----+-----+
|id_veiculo|5:13a|6:38a|7:32a|8:04a|9:07a|
+----------+-----+-----+-----+-----+-----+
|       298| null| null| null|    1|    1|
|       457|    1|    1| null| null| null|
|       315| null|    1|    1| null| null|
+----------+-----+-----+-----+-----+-----+



In [95]:
# Vamos converter o Spark Dataframe para Pandas Dataframe a fim de facilitar a visualização
pandasDF = df_pivot.toPandas()

In [96]:
pandasDF.head()

,id_veiculo,5:13a,6:38a,7:32a,8:04a,9:07a
0,298,NaN,NaN,NaN,1.0,1.0
1,457,1.0,1.0,NaN,NaN,NaN
2,315,NaN,1.0,1.0,NaN,NaN


In [97]:
# Define a query
# Número de entregas por id_veiculo
query = """
SELECT * FROM (
  SELECT id_veiculo, horario
  FROM tb_logistica
)
PIVOT (
  COUNT(*)
  FOR horario in (
    '5:13a', '6:38a', '7:32a', '8:04a', '9:07a'
  )
)
ORDER BY id_veiculo
"""

In [98]:
# Executa a query
spark.sql(query).show()

+----------+-----+-----+-----+-----+-----+
|id_veiculo|5:13a|6:38a|7:32a|8:04a|9:07a|
+----------+-----+-----+-----+-----+-----+
|       298| null| null| null|    1|    1|
|       315| null|    1|    1| null| null|
|       457|    1|    1| null| null| null|
+----------+-----+-----+-----+-----+-----+



## SQL Window Function Para Agregação ao Longo do Tempo

In [99]:
# Isso aqui é agregação por coluna (e por isso usamos o group by)
query = """
SELECT id_veiculo, COUNT(horario) AS numero_entregas
FROM tb_logistica
GROUP BY id_veiculo
"""

In [100]:
spark.sql(query).show()

+----------+---------------+
|id_veiculo|numero_entregas|
+----------+---------------+
|       298|              7|
|       457|              7|
|       315|              7|
+----------+---------------+



In [101]:
# Isso aqui é agregação por linha (e por isso usamos função window)
# Isso é útil quando os dados estão embaralhados e queremos organizar com base em um critério
# Por exemplo: 
# Para cada entrega (entrega 1, entrega 2, etc..), quais veículos fizeram a entrega primeiro em cada hora?
query = """
SELECT id_veiculo, entrega, horario,
ROW_NUMBER() OVER (PARTITION BY entrega ORDER BY horario) AS ranking
FROM tb_logistica
"""

In [102]:
spark.sql(query).show(21)

+----------+---------+-------+-------+
|id_veiculo|  entrega|horario|ranking|
+----------+---------+-------+-------+
|       457|Entrega 1|  5:04a|      1|
|       315|Entrega 1|  6:05a|      2|
|       298|Entrega 1|  7:58a|      3|
|       457|Entrega 2|  5:13a|      1|
|       315|Entrega 2|  6:14a|      2|
|       298|Entrega 2|  8:04a|      3|
|       457|Entrega 3|  5:27a|      1|
|       315|Entrega 3|  6:24a|      2|
|       298|Entrega 3|  8:17a|      3|
|       457|Entrega 4|  5:39a|      1|
|       315|Entrega 4|  6:38a|      2|
|       298|Entrega 4|  8:28a|      3|
|       457|Entrega 5|  5:47a|      1|
|       315|Entrega 5|  6:45a|      2|
|       298|Entrega 5|  8:33a|      3|
|       457|Entrega 6|  6:21a|      1|
|       315|Entrega 6|  6:56a|      2|
|       298|Entrega 6|  8:39a|      3|
|       457|Entrega 7|  6:38a|      1|
|       315|Entrega 7|  7:32a|      2|
|       298|Entrega 7|  9:07a|      3|
+----------+---------+-------+-------+



In [103]:
# Mesmo resultado anterior mas usando Função SparkSQL
df.withColumn("id", row_number().over(Window.partitionBy('entrega').orderBy('horario'))).show(21)

+----------+---------+-------+---+
|id_veiculo|  entrega|horario| id|
+----------+---------+-------+---+
|       457|Entrega 1|  5:04a|  1|
|       315|Entrega 1|  6:05a|  2|
|       298|Entrega 1|  7:58a|  3|
|       457|Entrega 2|  5:13a|  1|
|       315|Entrega 2|  6:14a|  2|
|       298|Entrega 2|  8:04a|  3|
|       457|Entrega 3|  5:27a|  1|
|       315|Entrega 3|  6:24a|  2|
|       298|Entrega 3|  8:17a|  3|
|       457|Entrega 4|  5:39a|  1|
|       315|Entrega 4|  6:38a|  2|
|       298|Entrega 4|  8:28a|  3|
|       457|Entrega 5|  5:47a|  1|
|       315|Entrega 5|  6:45a|  2|
|       298|Entrega 5|  8:33a|  3|
|       457|Entrega 6|  6:21a|  1|
|       315|Entrega 6|  6:56a|  2|
|       298|Entrega 6|  8:39a|  3|
|       457|Entrega 7|  6:38a|  1|
|       315|Entrega 7|  7:32a|  2|
|       298|Entrega 7|  9:07a|  3|
+----------+---------+-------+---+



In [104]:
# Define a query
# Para cada entrega mostra o horário da entrega anterior por id_veiculo
query = """
SELECT id_veiculo, entrega, horario, 
LAG(horario, 1) OVER (PARTITION BY id_veiculo ORDER BY horario) AS entrega_anterior 
FROM tb_logistica
"""

In [105]:
spark.sql(query).show(21)

+----------+---------+-------+----------------+
|id_veiculo|  entrega|horario|entrega_anterior|
+----------+---------+-------+----------------+
|       298|Entrega 1|  7:58a|            null|
|       298|Entrega 2|  8:04a|           7:58a|
|       298|Entrega 3|  8:17a|           8:04a|
|       298|Entrega 4|  8:28a|           8:17a|
|       298|Entrega 5|  8:33a|           8:28a|
|       298|Entrega 6|  8:39a|           8:33a|
|       298|Entrega 7|  9:07a|           8:39a|
|       315|Entrega 1|  6:05a|            null|
|       315|Entrega 2|  6:14a|           6:05a|
|       315|Entrega 3|  6:24a|           6:14a|
|       315|Entrega 4|  6:38a|           6:24a|
|       315|Entrega 5|  6:45a|           6:38a|
|       315|Entrega 6|  6:56a|           6:45a|
|       315|Entrega 7|  7:32a|           6:56a|
|       457|Entrega 1|  5:04a|            null|
|       457|Entrega 2|  5:13a|           5:04a|
|       457|Entrega 3|  5:27a|           5:13a|
|       457|Entrega 4|  5:39a|          

In [106]:
# Define a query
# Para cada entrega mostra o horário da próxima entrega por id_veiculo
query = """
SELECT 
id_veiculo, entrega, horario, 
LEAD(horario, 1) OVER (PARTITION BY id_veiculo ORDER BY horario) AS proxima_entrega 
FROM tb_logistica
"""

In [107]:
# Executa a query e mostra o resultado
spark.sql(query).show(21)

+----------+---------+-------+---------------+
|id_veiculo|  entrega|horario|proxima_entrega|
+----------+---------+-------+---------------+
|       298|Entrega 1|  7:58a|          8:04a|
|       298|Entrega 2|  8:04a|          8:17a|
|       298|Entrega 3|  8:17a|          8:28a|
|       298|Entrega 4|  8:28a|          8:33a|
|       298|Entrega 5|  8:33a|          8:39a|
|       298|Entrega 6|  8:39a|          9:07a|
|       298|Entrega 7|  9:07a|           null|
|       315|Entrega 1|  6:05a|          6:14a|
|       315|Entrega 2|  6:14a|          6:24a|
|       315|Entrega 3|  6:24a|          6:38a|
|       315|Entrega 4|  6:38a|          6:45a|
|       315|Entrega 5|  6:45a|          6:56a|
|       315|Entrega 6|  6:56a|          7:32a|
|       315|Entrega 7|  7:32a|           null|
|       457|Entrega 1|  5:04a|          5:13a|
|       457|Entrega 2|  5:13a|          5:27a|
|       457|Entrega 3|  5:27a|          5:39a|
|       457|Entrega 4|  5:39a|          5:47a|
|       457|E

## Usando Partições com Spark SQL
* A função **over()** no Spark SQL corresponde a cláusula **OVER** em SQL

In [108]:
# Abre a janela nos dados
janela = Window.partitionBy('id_veiculo').orderBy('horario')

In [109]:
type(janela)

pyspark.sql.window.WindowSpec

In [110]:
# Aplica o Lead (desloca os dados no tempo) sobre (over) a janela (window)
dfx = df.withColumn('proxima_entrega', lead('horario', 1).over(janela))

In [111]:
dfx.show(21)

+----------+---------+-------+---------------+
|id_veiculo|  entrega|horario|proxima_entrega|
+----------+---------+-------+---------------+
|       298|Entrega 1|  7:58a|          8:04a|
|       298|Entrega 2|  8:04a|          8:17a|
|       298|Entrega 3|  8:17a|          8:28a|
|       298|Entrega 4|  8:28a|          8:33a|
|       298|Entrega 5|  8:33a|          8:39a|
|       298|Entrega 6|  8:39a|          9:07a|
|       298|Entrega 7|  9:07a|           null|
|       315|Entrega 1|  6:05a|          6:14a|
|       315|Entrega 2|  6:14a|          6:24a|
|       315|Entrega 3|  6:24a|          6:38a|
|       315|Entrega 4|  6:38a|          6:45a|
|       315|Entrega 5|  6:45a|          6:56a|
|       315|Entrega 6|  6:56a|          7:32a|
|       315|Entrega 7|  7:32a|           null|
|       457|Entrega 1|  5:04a|          5:13a|
|       457|Entrega 2|  5:13a|          5:27a|
|       457|Entrega 3|  5:27a|          5:39a|
|       457|Entrega 4|  5:39a|          5:47a|
|       457|E

In [112]:
# Mesmo exemplo anterior mas em uma linha de código
df_dot = df.withColumn('proxima_entrega', lead('horario', 1)
.over(Window.partitionBy('id_veiculo')
.orderBy('horario'))).show(21)

+----------+---------+-------+---------------+
|id_veiculo|  entrega|horario|proxima_entrega|
+----------+---------+-------+---------------+
|       298|Entrega 1|  7:58a|          8:04a|
|       298|Entrega 2|  8:04a|          8:17a|
|       298|Entrega 3|  8:17a|          8:28a|
|       298|Entrega 4|  8:28a|          8:33a|
|       298|Entrega 5|  8:33a|          8:39a|
|       298|Entrega 6|  8:39a|          9:07a|
|       298|Entrega 7|  9:07a|           null|
|       315|Entrega 1|  6:05a|          6:14a|
|       315|Entrega 2|  6:14a|          6:24a|
|       315|Entrega 3|  6:24a|          6:38a|
|       315|Entrega 4|  6:38a|          6:45a|
|       315|Entrega 5|  6:45a|          6:56a|
|       315|Entrega 6|  6:56a|          7:32a|
|       315|Entrega 7|  7:32a|           null|
|       457|Entrega 1|  5:04a|          5:13a|
|       457|Entrega 2|  5:13a|          5:27a|
|       457|Entrega 3|  5:27a|          5:39a|
|       457|Entrega 4|  5:39a|          5:47a|
|       457|E

## Parse de Data Para Agregação ao Longo do Tempo


In [113]:
# Define o time parser policy
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

In [114]:
# Cria a janela
window = Window.partitionBy('id_veiculo').orderBy('horario')

In [115]:
# Agregação por linha para calcular a diferença entre os horários
dot_df = df.withColumn('tempo_proxima_entrega', 
                       (unix_timestamp(lead('horario', 1).over(window),'H:m') 
                        - unix_timestamp('horario', 'H:m'))/60).show(21)

+----------+---------+-------+---------------------+
|id_veiculo|  entrega|horario|tempo_proxima_entrega|
+----------+---------+-------+---------------------+
|       298|Entrega 1|  7:58a|                  6.0|
|       298|Entrega 2|  8:04a|                 13.0|
|       298|Entrega 3|  8:17a|                 11.0|
|       298|Entrega 4|  8:28a|                  5.0|
|       298|Entrega 5|  8:33a|                  6.0|
|       298|Entrega 6|  8:39a|                 28.0|
|       298|Entrega 7|  9:07a|                 null|
|       315|Entrega 1|  6:05a|                  9.0|
|       315|Entrega 2|  6:14a|                 10.0|
|       315|Entrega 3|  6:24a|                 14.0|
|       315|Entrega 4|  6:38a|                  7.0|
|       315|Entrega 5|  6:45a|                 11.0|
|       315|Entrega 6|  6:56a|                 36.0|
|       315|Entrega 7|  7:32a|                 null|
|       457|Entrega 1|  5:04a|                  9.0|
|       457|Entrega 2|  5:13a|                

In [116]:
# Define a query
# Calculamos aqui a diferença de tempo de uma entrega para outra por id_veiculo (partição)
query = """
SELECT *, 
(UNIX_TIMESTAMP(LEAD(horario, 1) OVER (PARTITION BY id_veiculo ORDER BY horario),'H:m') 
- UNIX_TIMESTAMP(horario, 'H:m'))/60 AS tempo_proxima_entrega
FROM tb_logistica 
"""

In [117]:
sql_df = spark.sql(query)

In [118]:
sql_df.show(21)

+----------+---------+-------+---------------------+
|id_veiculo|  entrega|horario|tempo_proxima_entrega|
+----------+---------+-------+---------------------+
|       298|Entrega 1|  7:58a|                  6.0|
|       298|Entrega 2|  8:04a|                 13.0|
|       298|Entrega 3|  8:17a|                 11.0|
|       298|Entrega 4|  8:28a|                  5.0|
|       298|Entrega 5|  8:33a|                  6.0|
|       298|Entrega 6|  8:39a|                 28.0|
|       298|Entrega 7|  9:07a|                 null|
|       315|Entrega 1|  6:05a|                  9.0|
|       315|Entrega 2|  6:14a|                 10.0|
|       315|Entrega 3|  6:24a|                 14.0|
|       315|Entrega 4|  6:38a|                  7.0|
|       315|Entrega 5|  6:45a|                 11.0|
|       315|Entrega 6|  6:56a|                 36.0|
|       315|Entrega 7|  7:32a|                 null|
|       457|Entrega 1|  5:04a|                  9.0|
|       457|Entrega 2|  5:13a|                

# Fim